# Integration

In [57]:
import pennylane as qml
from pennylane.optimize import AdamOptimizer
from pennylane.math import toarray
from pennylane import numpy as np
import math

import matplotlib.pyplot as plt

### Data Generation

In [2]:
data = np.load('toy_data_10e4.npz')

In [10]:
Xdata = data['xdata']
print(Xdata[:5])
print(Xdata[4:])
print(len(Xdata[0]))

[[2.55258737 0.41757623 2.5810235  0.        ]
 [1.05945455 2.73902844 2.59982768 0.01002004]
 [2.71202945 3.38179615 1.0272925  0.02004008]
 [2.21421541 0.12472855 1.58366213 0.03006012]
 [1.40135874 2.4821793  0.11875834 0.04008016]]
[[1.40135874 2.4821793  0.11875834 0.04008016]
 [0.16315241 2.51735018 2.47341997 0.0501002 ]
 [2.42817168 2.90637455 1.44798689 0.06012024]
 ...
 [2.26058831 2.62355914 0.18093676 4.97995992]
 [2.91426381 3.41772047 2.93088925 4.98997996]
 [0.1022243  2.0529486  1.72946409 5.        ]]
4


In [31]:
Ydata = data['ydata']
print(Ydata)

[-0.03413077  0.00653628 -0.83397146 ...  0.9999308  -0.8789377
 -0.79724696]


In [5]:
np.random.seed(32)

In [32]:
def f(x):
    x1 = x[0]
    x2 = x[1]
    x3 = x[2]
    a0 = x[3]
    return np.cos(x1 + 2*x2 + 1/2*x3 + a0)

print(f(Xdata[0]))

-0.03413076931141696


### Circuit Model

In [50]:
dev_stochastic = qml.device("lightning.qubit", wires = 10)

@qml.qnode(dev_stochastic)
def Ansatz(thetas, phis, x, num_layers):
    """
    Quantum Circuit Model

    INPUT
    thetas : array of theta parameters (theta_0, theta_1, theta_2, theta_3)
    phis : array of phi for the last rotation gate
    xdim : dimension of variables
    num_layers : layers we will append for the circuit

    OUTPUT
    Expectation values with PauliZ measure
    """
    
    xdim = len(x)
    num_qubits = math.ceil(xdim / 2)
    idx = 0

    params = 0
    params += thetas
    
    param_index = 1
    for _ in range(num_layers):
        for j in range(xdim):
            params[param_index] = thetas[param_index] * x[j]
            param_index += 5

    for _ in range(num_layers):
        k = 0
        for i in range(xdim):
            qml.RY(params[idx], wires = int(k))
            qml.RZ(params[idx+1], wires = int(k), id = f'x{i}')
            qml.RZ(params[idx+2], wires = int(k))
            qml.RY(params[idx+3], wires = int(k))
            qml.RZ(params[idx+4], wires = int(k))
            idx += 5
            k+=1/2

        if num_qubits > 1:
            for q in range(0,num_qubits-1,1):
                qml.CZ([q, q+1])
            if num_qubits > 2:
                qml.CZ([num_qubits-1, 0])
    
    if num_qubits > 1:
            for i in range(num_qubits):
                qml.RY(phis[i], wires=i, id = f"phi{i}")
    
    obs = qml.PauliZ(0)
    for i in range(num_qubits-1):
         obs = obs @ qml.PauliZ(i+1)

    return qml.expval(obs)

### Loss function

In [58]:
def parameter_shift_whole(thetas, phis, x, num_layers):
    """
    Basic Parameter shift rule for each x_i i= 0,1, ..., xdim
   
    INPUT
    qnode :  circuit ansatz we designed
    parmas : (array) of parameters we put in the circuit
    x_i : (int) ${{\partial f(\theta)} \over {\partial x_i}}$
    phis : (array) the last parameters of the circuit
    ndim : (int) dimenstion of $\vec{x}$
    num_layers : (int) number of layer we make for the model

    OUTPUT
    expectation value where we apply the Basic PSR for the circuit
    """
    thetas_n = toarray(thetas)
    phis_n = toarray(phis)

    shifted_thetas = thetas_n.copy()
    shifted_phis = phis_n.copy()
    
    shifted_thetas[:] += np.pi/2
    shifted_phis[:] += np.pi/2
    forward = Ansatz(shifted_thetas, shifted_phis, x, num_layers)  # forward evaluation

    shifted_thetas[:] -= np.pi
    shifted_phis[:] -= np.pi
    backward = Ansatz(shifted_thetas, shifted_phis, x, num_layers) # backward evaluation

    return 0.5 * (forward - backward)

In [46]:
def loss_function(thetas, phis, Xdata, num_layers, target):
    """
    loss function

    INPUT
    

    OUTPUT
    loss with MSE
    """

    loss = 0
    
    for i in range(len(Xdata)):
        estimated_val = parameter_shift_whole(thetas, phis, Xdata[i], num_layers)
        target_val = target[i]

        loss += ((estimated_val - target_val) ** 2)

    return loss / len(Xdata)

xdim = len(Xdata[0])
num_layers = 1
num_qubits = math.ceil(xdim / 2)

thetas = np.random.uniform(size=xdim*num_layers*5, requires_grad=True)
phis = np.random.uniform(size = num_qubits, requires_grad =True)
    
loss_function(thetas, phis, Xdata, num_layers, Ydata)

0.565075233657859

### Model Runnig

In [33]:
def model_running(thetas, phis, Xdata, num_layers):
    """
    Model running with given set of data.

    INPUT
    params : array of parameters 
    phis : array of last RY rotation
    Xdata : values of xdata

    OUTPU
    Expectaion values(Integration value ???), Estimated value(derivative of the circuit)
    """
    exepctation_values = []
    estimated_values = []
    
    params = 0
    params += thetas
   
    for i in range(len(Xdata)):

        expval = Ansatz(thetas, phis, Xdata[i], num_layers)
        exepctation_values.append(expval)

        estimated_val = parameter_shift_whole(thetas, phis, Xdata[i], num_layers)
        estimated_values.append(estimated_val)
        
    return np.array(exepctation_values), np.array(estimated_values)

In [61]:
def accuracy_score(y_true, y_pred):
    """
    Accuracy score. Evaluating the model with the label comparing.
    
    INPUT
    y_true : Targets(Answers)
    y_predicted : Predictions(labels wihch model has given)

    OUTPUT
    the fraction of correctly classified samples
    """
    score = 0
    for i in range(len(y_true)):
        score += y_pred == y_true 

    return score.sum() / len(y_true)

In [35]:
def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data
    
    INPUT
    inputs : input data
    targets : targets
    batch_size : size of the batch, the number of datas in one batch

    Returns
    one batch of input data of length `batch_size`, one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

----

In [62]:
data = np.load('toy_data_10e4.npz')
Xdata = data['xdata']
Ydata = data['ydata']

#Dividing the trainin data and test data
train_data = Xdata[:750]
train_target = Ydata[:750]
test_data = Xdata[750:]
test_target = Ydata[750:]

# Checking the data
xdim = len(Xdata[0])
num_qubits = math.ceil(xdim / 2)

# Trainnig option settings
num_layers = 1
epochs = 25
batch_size = 50
eta = 0.1

# Using the Optimizer
opt = AdamOptimizer(stepsize = eta)

# Initializing random parameters for the circuit
thetas = np.random.uniform(size=xdim*num_layers*5, requires_grad=True)
phis = np.random.uniform(size = num_qubits, requires_grad =True)

### Evaluating the qNN
# Running the model with test data
expvals_train, predicted_train = model_running(thetas, phis, train_data, num_layers)
accuracy_train = accuracy_score(train_target, predicted_train)

# Running the model with the test data
expvals_test, predicted_test = model_running(thetas, phis, test_data, num_layers)
accuracy_test = accuracy_score(test_target, predicted_test)

# Saving predictions with random weights for comparison 
initial_predictions = predicted_test

loss = loss_function(thetas, phis, test_data, num_layers, test_target)

loss_list, accuracy_train_list, accuracy_test_list = [], [], []
loss_list.append(loss)
accuracy_train_list.append(accuracy_train)
accuracy_test_list.append(accuracy_test)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test
    )
)

for it in range(epochs):
    for Xbatch, ybatch in iterate_minibatches(train_data, train_target, batch_size=batch_size):
        thetas, phis, _, _, _ = opt.step(loss_function, thetas, phis, Xbatch, num_layers, ybatch)

    predicted_train, expvals_train = model_running(thetas, phis, test_data, num_layers)
    accuracy_train = accuracy_score(train_target, predicted_train)
    loss = loss_function(Ansatz, thetas, phis, test_data, xdim, num_layers, test_target)

    predicted_test, expvals_test = model_running(thetas, phis, test_data, num_layers)
    accuracy_test = accuracy_score(test_target, predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )

    loss_list.append(loss)
    accuracy_train_list.append(accuracy_train)
    accuracy_test_list.append(accuracy_test)

Epoch:  0 | Cost: 0.625248 | Train accuracy: -9711.480171 | Test Accuracy: -2108.816133


/opt/homebrew/lib/python3.11/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


ValueError: operands could not be broadcast together with shapes (9250,) (750,) 